# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
import pandas as pd
import numpy as np
import re #provides regular expression matching operations
from scipy.stats import ttest_ind
from scipy import stats

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
university_towns = open('university_towns.txt').readlines()
university_towns[0:30]

['Alabama[edit]\n',
 'Auburn (Auburn University)[1]\n',
 'Florence (University of North Alabama)\n',
 'Jacksonville (Jacksonville State University)[2]\n',
 'Livingston (University of West Alabama)[2]\n',
 'Montevallo (University of Montevallo)[2]\n',
 'Troy (Troy University)[2]\n',
 'Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]\n',
 'Tuskegee (Tuskegee University)[5]\n',
 'Alaska[edit]\n',
 'Fairbanks (University of Alaska Fairbanks)[2]\n',
 'Arizona[edit]\n',
 'Flagstaff (Northern Arizona University)[6]\n',
 'Tempe (Arizona State University)\n',
 'Tucson (University of Arizona)\n',
 'Arkansas[edit]\n',
 'Arkadelphia (Henderson State University, Ouachita Baptist University)[2]\n',
 'Conway (Central Baptist College, Hendrix College, University of Central Arkansas)[2]\n',
 'Fayetteville (University of Arkansas)[7]\n',
 'Jonesboro (Arkansas State University)[8]\n',
 'Magnolia (Southern Arkansas University)[2]\n',
 'Monticello (University of Arkansas at Montice

In [6]:
states = [line.split('[edit]')[0] for line in university_towns if '[edit]' in line]
print(states)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [7]:
university_towns = [line.split('(')[0] for line in university_towns]
university_towns[0:15]

['Alabama[edit]\n',
 'Auburn ',
 'Florence ',
 'Jacksonville ',
 'Livingston ',
 'Montevallo ',
 'Troy ',
 'Tuscaloosa ',
 'Tuskegee ',
 'Alaska[edit]\n',
 'Fairbanks ',
 'Arizona[edit]\n',
 'Flagstaff ',
 'Tempe ',
 'Tucson ']

In [8]:
university_towns = '\n'.join(university_towns)
#states_pattern = '\[edit\]\n|'.join(states) + '\[edit\]\n'
university_towns[0:100]

'Alabama[edit]\n\nAuburn \nFlorence \nJacksonville \nLivingston \nMontevallo \nTroy \nTuscaloosa \nTuskegee \nA'

In [9]:
s2 = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h']) #split return a Series of lists

s2.str.split('_',expand=True) # this is a series

,0,1,2
0,a,b,c
1,c,d,e
2,NaN,None,None
3,f,g,h


In [10]:
def get_list_of_university_towns(): #solution 1 everything together
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame is:
#     DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State","RegionName"]  )'''
    
    file = open('university_towns.txt').readlines()
    #file = ''.join(file).replace('*','\n').split('\n')
    states = [line.split('[edit]')[0] for line in file if '[edit]' in line]
    file = [x.split(' (')[0] for x in file]

    file = '\n'.join(file) + '\n' #converts the list to the string, adds a \n between
    #each two elements of the list and finally adds a \n at the end of the string
    states_pattern = '\[edit\]\n|'.join(states) + '\[edit\]\n'

    file = re.split(states_pattern, file)[1:]
    schools = []

    for state in range(len(states)):
        for school in file[state].split('\n'):
            if school != '\n' and school != '':
                schools.append({"State":states[state], "RegionName":school})
    df = pd.DataFrame(schools)
    
    return df.iloc[:,::-1]
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [11]:
def get_list_of_university_towns(): # my version
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State","RegionName"]  )'''
  
# Instructor's pseudo code    
# read the file as lines
# lst=[]
# state=''
# region=''
# for each line
#     if line contains '[ed' then the line has a state
#             state = get the state by removing everything from '[' to end
#     else
#         if the line has a '('
#             region = get region by removing everything from '(' to end 
#         else
#             region =  line with \n removed
#         endif
#     endif
#     add state,region to lst
# endfor

    file = open('university_towns.txt').readlines() #with open('university_towns.txt') as file:
    data = [] #a list
#    state=''
#    region=''
    for line in file:
        if '[edit]' in line:
            state = line.split('[edit]')[0]
        #data.append(line[:-1]) # :-1 means minus two characters from the end and this removes the \ns
        else:
            if '(' in line:
                region = line.split(' (')[0]
            else:
                region = line.split(' (')[0]
                
#        if region != '':
            data.append([state,region])
                    
    df= pd.DataFrame(data, columns=["State", "RegionName"])
    return df.head() #state_college_df
        
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [18]:
def get_list_of_university_towns():
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State","RegionName"]  )'''
    
    list_of_uni_towns=pd.read_table('university_towns.txt', header=None, delim_whitespace=True, names=('State', 'RegionName'))
    return list_of_uni_towns.head #"ANSWER"

get_list_of_university_towns()

<bound method NDFrame.head of                 State      RegionName
0       Alabama[edit]             NaN
1              Auburn         (Auburn
2            Florence     (University
3        Jacksonville   (Jacksonville
4          Livingston     (University
5          Montevallo     (University
6                Troy           (Troy
7          Tuscaloosa     (University
8            Tuskegee       (Tuskegee
9        Alaska[edit]             NaN
10          Fairbanks     (University
11      Arizona[edit]             NaN
12          Flagstaff       (Northern
13              Tempe        (Arizona
14             Tucson     (University
15     Arkansas[edit]             NaN
16        Arkadelphia      (Henderson
17             Conway        (Central
18       Fayetteville     (University
19          Jonesboro       (Arkansas
20           Magnolia       (Southern
21         Monticello     (University
22       Russellville       (Arkansas
23             Searcy        (Harding
24   California[edit

In [19]:
def get_recession_start(): 
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    gdp = gdp[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']].ix[214:].reset_index(drop=True)

    gdp = gdp.rename(columns={'Unnamed: 4':'Quarters'})
    gdp['DIFF'] = gdp['GDP in billions of chained 2009 dollars.1'].diff(periods=1)#.apply(lambda x: np.where(x>=1, 1, 0))
    gdp['recess' ] = np.nan


    gdp['recess' ] = np.nan
    gdp.loc[ ((gdp.DIFF<0) & (gdp.DIFF.shift(-1)<0)), 'recess' ] = 1 #& (gdp.DIFF.shift(1)>0)
    ans = gdp.iloc[34]['Quarters']

    return ans #gdp.head(50)

get_recession_start()


'2008q3'

In [21]:
def get_recession_start(): # solution 1
#     '''Returns the year and quarter of the recession start time as a
#    string value in a format such as 2005q3'''
   
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[['Unnamed: 4','Unnamed: 5']]
    gdp = gdp[-66:]
    gdp = gdp.rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP'})
    gdp['GDP+1'] = list(gdp['GDP'][1:].values) + [np.nan]
    gdp['GDP+2'] = list(gdp['GDP+1'][1:].values) + [np.nan]
    gdp['diff1'] = gdp['GDP+1'] - gdp['GDP']
    gdp['diff2'] = gdp['GDP+2'] - gdp['GDP+1']
    c1 = gdp['diff1'] < 0
    c2 = gdp['diff2'] < 0
    gdp[c1 & c2]
    recession = gdp[c1 & c2]
    return recession['Quarter'].values[0]

get_recession_start()

'2008q3'

In [22]:
def get_recession_start():
#     '''Returns the year and quarter of the recession start time as a 
#     string value in a format such as 2005q3'''

    x = pd.ExcelFile('gdplev.xls')
    gdp = x.parse(skiprows=7)#skiprows=17,skip_footer=(38)) #parses an Excel sheet into a dataframe
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp = gdp.loc[212:]
    gdp.columns = ['Quarter','GDP']
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    quarters = []
    for i in range(len(gdp) - 2):
        if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
            quarters.append(gdp.iloc[i][0])
    return quarters[0]
    
get_recession_start()

'2008q3'

In [23]:
def get_recession_end(): #Solution 1 we are looking for a +ve diff where the diff b4 is also +ve and the
    # one and 2 ones b4 that are -ve. So recession end is the 2nd +ve diff after 2 consecutive -ves.
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    gdp = gdp[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']].ix[214:].reset_index(drop=True)

    gdp = gdp.rename(columns={'Unnamed: 4':'Quarters'})
    gdp['DIFF'] = gdp['GDP in billions of chained 2009 dollars.1'].diff(periods=1)#.apply(lambda x: np.where(x>=1, 1, 0))
    gdp['recess' ] = np.nan


    gdp['recess' ] = np.nan
    gdp.loc[ ((gdp.DIFF>0) & (gdp.DIFF.shift(1)>0)) & (gdp.DIFF.shift(2)<0) & (gdp.DIFF.shift(3)<0), 'recess' ] = 1 #

    return gdp['Quarters'][39]

get_recession_end()

'2009q4'

In [24]:
def get_recession_end(): #Solution 1
#     '''Returns the year and quarter of the recession end time as a
#    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[['Unnamed: 4','Unnamed: 5']]
    gdp = gdp[-66:]
    gdp = gdp.rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP'})
    gdp['GDP-1'] = [np.nan] + list(gdp['GDP'][0:-1].values)
    gdp['GDP-2'] = [np.nan] + list(gdp['GDP-1'][0:-1].values)
    gdp['diff1'] = gdp['GDP'] - gdp['GDP-1']
    gdp['diff2'] = gdp['GDP-1'] - gdp['GDP-2']
    c1 = gdp['diff1'] > 0
    c2 = gdp['diff2'] > 0
    c3 = gdp['Quarter'] > get_recession_start()
    gdp[c1 & c2 & c3]
    end = gdp[c1 & c2 & c3]
    return end['Quarter'].values[0]

get_recession_end()

'2009q4'

In [25]:
def get_recession_end():
#     '''Returns the year and quarter of the recession end time as a 
#     string value in a format such as 2005q3'''
    x = pd.ExcelFile('gdplev.xls')
    gdp = x.parse(skiprows=7)#skiprows=17,skip_footer=(38))
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp = gdp.loc[212:]
    gdp.columns = ['Quarter','GDP']
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    quarters = []
    for i in range(len(gdp) - 2):
        if (gdp.iloc[i][1] < gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] < gdp.iloc[i+2][1]):
            quarters.append(gdp.iloc[i][0])
    return '2009q4'

get_recession_end()  
    

'2009q4'

In [26]:
def get_recession_bottom():
#     '''Returns the year and quarter of the recession bottom time as a
#    string value in a format such as 2005q3'''
   
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[['Unnamed: 4','Unnamed: 5']]
    gdp = gdp[-66:]
    gdp = gdp.rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP'})
    c1 = gdp['Quarter'] >= get_recession_start()
    c2 = gdp['Quarter'] <= get_recession_end()
    gdp = gdp[c1 & c2]

    return gdp[gdp['GDP'] == gdp['GDP'].min()].values[0][0]

get_recession_bottom()

'2009q2'

In [27]:
def convert_housing_data_to_quarters():
#     '''Converts the housing data to quarters and returns it as mean
#    values in a dataframe. This dataframe should be a dataframe with
#    columns for 2000q1 through 2016q3, and should have a multi-index
#    in the shape of ["State","RegionName"].
   
#    Note: Quarters are defined in the assignment description, they are
#    not arbitrary three month periods.
   
#    The resulting dataframe should have 67 columns, and 10,730 rows.
#    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    towns = data[['State','RegionName']]
    for yr in range(2000,2017):
        for q in range(1,12,3):
            if yr == 2016 and q == 4:
                break
            m1 = '{}-{:02d}'.format(yr,q)
            m2 = '{}-{:02d}'.format(yr,q+1)
            m3 = '{}-{:02d}'.format(yr,q+2)
            quarter = '{}q{}'.format(yr,q)
            towns[quarter] = data[[m1,m2,m3]].mean(axis=1)
   
    return towns

convert_housing_data_to_quarters()

C:\Users\Cyrus\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,State,RegionName,2000q1,2000q4,2000q7,2000q10,2001q1,2001q4,2001q7,2001q10,...,2013q10,2014q1,2014q4,2014q7,2014q10,2015q1,2015q4,2015q7,2015q10,2016q1
0,NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.924333e+05,5.031333e+05,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05
1,CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,...,4.719333e+05,4.842667e+05,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05
2,IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,...,1.858667e+05,1.909000e+05,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05
3,PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,...,1.124667e+05,1.137333e+05,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05
4,AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,...,1.599000e+05,1.632333e+05,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05
5,NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,...,1.625667e+05,1.660000e+05,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05
6,CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,...,4.664667e+05,4.740000e+05,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05
7,TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,...,1.045333e+05,1.053333e+05,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05
8,CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,...,6.456333e+05,6.611667e+05,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05
9,FL,Jacksonville,8.860000e+04,8.970000e+04,9.170000e+04,9.310000e+04,9.440000e+04,9.560000e+04,9.706667e+04,9.906667e+04,...,1.155667e+05,1.189333e+05,1.207667e+05,1.217333e+05,1.231667e+05,1.241667e+05,1.269000e+05,1.301333e+05,1.320000e+05,1.339667e+05


In [32]:
def get_recession_data():
    '''Returns the year and quarter of the recession start, end, and bottom time as a 
    string value in a format such as 2005q3'''
    
    filename = 'gdplev.xls'
    
    economy_df = pd.read_excel(filename,skiprows=5).iloc[2:,[4,6]]
    
    economy_df = economy_df.rename(index = str, columns = {'Unnamed: 4':'Quarter',
                                             'GDP in billions of chained 2009 dollars.1':'GDP (billions)'})

    economy_df = economy_df[economy_df.Quarter >= '2000q1']

    economy_df['Next Quarter GDP'] = list(economy_df['GDP (billions)'].iloc[1:]) + [np.NAN]
    economy_df['Two Quarters GDP'] = list(economy_df['GDP (billions)'].iloc[2:]) + 2*[np.NAN]

    economy_df['Recession Begin'] = (economy_df['GDP (billions)'] > 
                                     economy_df['Next Quarter GDP']) & (
                                    economy_df['Next Quarter GDP'] > 
                                    economy_df['Two Quarters GDP'])

    recession_begin = economy_df[economy_df['Recession Begin'] == True]

    economy_df['Last Quarter GDP'] = [np.NAN] + list(economy_df['GDP (billions)'].iloc[:-1])
    
    economy_df['Two Quarters Ago GDP'] = 2*[np.NAN] + list(economy_df['GDP (billions)'].iloc[:-2])
    
    economy_df['Recession End'] = (economy_df['GDP (billions)'] > 
                                     economy_df['Last Quarter GDP']) & (
                                    economy_df['Last Quarter GDP'] > 
                                    economy_df['Two Quarters Ago GDP']) & (
                                    economy_df['Quarter'] > recession_begin['Quarter'].values[0])
    recession_end = economy_df[(economy_df['Recession End'] == True)]
    recession_end['Quarter'].values[0]

    economy_df = economy_df[(economy_df.Quarter >= recession_begin['Quarter'].values[0]) & (economy_df.Quarter <= recession_end['Quarter'].values[0])]
    
    recession_bottom = economy_df[economy_df['GDP (billions)'] == economy_df['GDP (billions)'].min()]

    recession_bottom['Quarter'].values[0]
    
    return recession_begin['Quarter'].values[0], recession_end['Quarter'].values[0], recession_bottom['Quarter'].values[0]

get_recession_data()

('2008q2', '2009q4', '2009q2')

In [33]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recession_begin, recession_end, recession_bottom = get_recession_data()
    
    university_towns = get_list_of_university_towns()
    
    housing_df = convert_housing_data_to_quarters()
    
    prices_begin = housing_df[recession_begin]
    prices_end = housing_df[recession_end]
    
    housing_df['ratio'] = prices_begin/prices_end #The higher this ratio, the worse
    
    university_towns = university_towns.set_index(['State', 'RegionName'])
    
    ratio_college = housing_df.loc[list(university_towns.index)]['ratio'].dropna()
    
    ratio_not_college_indices = set(housing_df.index) - set(ratio_college.index)
    
    ratio_not_college = housing_df.loc[list(ratio_not_college_indices),:]['ratio'].dropna()
        
    statistic, p_value = tuple(stats.ttest_ind(ratio_college, ratio_not_college))
    
    outcome = statistic < 0
    
    different = p_value < 0.05
    
    better = ["non-university town", "university town"]
    
    return (different, p_value, better[outcome])

run_ttest()

C:\Users\Cyrus\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyError: '2008q2'

In [28]:
def run_ttest():
#     '''First creates new data showing the decline or growth of housing prices
#    between the recession start and the recession bottom. Then runs a ttest
#    comparing the university town values to the non-university towns values,
#    return whether the alternative hypothesis (that the two groups are the same)
#    is true or not as well as the p-value of the confidence.
   
#    Return the tuple (different, p, better) where different=True if the t-test is
#    True at a p<0.01 (we reject the null hypothesis), or different=False if
#    otherwise (we cannot reject the null hypothesis). The variable p should
#    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
#    value for better should be either "university town" or "non-university town"
#    depending on which has a lower mean price ratio (which is equivilent to a
#    reduced market loss).'''
   
    from scipy.stats import ttest_ind
 
    start = get_recession_start()
    bottom = get_recession_bottom()
    prices = convert_housing_data_to_quarters()
 
    price = pd.DataFrame()
    price['ratio'] = prices[start] / prices[bottom]
    price = price.reset_index()
    price = price.dropna()
 
    university = get_list_of_university_towns()
 
    def is_university_town(row):
        c1 =  university['State'] == row['State']
        c2 =  university['RegionName'] == row['RegionName']
        return len(university[c1 & c2]) > 0
 
    price['uni'] = price.apply(is_university_town,axis=1)
 
    prices_uni = price[price['uni']]
    prices_notuni = price[price['uni'] == False]
 
    ttest = ttest_ind(prices_uni['ratio'], prices_notuni['ratio'])
 
    hyp = ttest.pvalue < 0.01
    better = 'university town'
    if ttest.statistic > 0:
        better = 'non-university town'
    result = (hyp, ttest.pvalue, better)
 
    return result

run_ttest()

C:\Users\Cyrus\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyError: '2008q3'